In [ ]:
# Install required packages and download data
%pip install -q -r requirements.txt
!sh data_get.sh

# NOTE: Changes some values meaningfully
# !sh data_wrangle.sh

In [ ]:
import duckdb
import pandas as pd
from collections import defaultdict
from os import listdir 

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# trying in-memory database, swap for a file-based db if needed
# %sql duckdb:///cerulean.db
%sql duckdb:///:memory:

In [ ]:
# Load the column data_types
raw_col = pd.read_csv('columns.csv', delimiter='\t')
raw_col['param'] = '\'' + raw_col['column'] + '\': \'' + raw_col['data_type']  + '\''

# Aggregate the column data_types for each schema.table
df_col = raw_col.groupby(['schema', 'table'])['param'].aggregate(", ".join).reset_index()
df_col['param'] = 'columns={' + df_col['param'] + '}'

# Pass into a dict, keys are (schema, table) tuples
col = df_col.set_index(['schema', 'table']).to_dict()['param']

# Example calls: set `columns` if known, otherwise use AUTO_DETECT
# print(col.get(('mimiciv_hosp', 'pharmacy'), 'AUTO_DETECT=TRUE'))
# print(col.get(('bad_schema', 'missing'), 'AUTO_DETECT=TRUE'))


In [ ]:
# Specify NULLSTR for problematic tables
nullstr = {}
# nullstr[('mimiciv_hosp', 'discharge')] = ', NULLSTR="___"'
# nullstr['mimiciv_hosp.emar_detail'] = ', NULLSTR="___"'
# nullstr['mimiciv_hosp.pharmacy'] = ', NULLSTR="___"'

In [ ]:
# Define a function to create a schema for each data source
# and a table for each file (*.csv.gz) in its directory

def create_schema_and_tables(schema, path):
    files = listdir(path)
    %sql CREATE SCHEMA IF NOT EXISTS {schema};
    for file in files:
        if file.endswith('.csv.gz'):
            table = file.split('.')[0]
            param = col.get((schema, table), 'AUTO_DETECT=TRUE') \
                  + nullstr.get((schema, table), '')
            # print(param)
            %sql DROP TABLE IF EXISTS {schema}.{table};
            %sql CREATE TABLE {schema}.{table} AS SELECT * FROM read_csv('{path}{file}', header=True, {param} );
    %sql result << SELECT table_schema, table_name FROM information_schema.tables WHERE table_schema = '{schema}';
    return result

In [ ]:
# Import MIMIC-IV Hospital
schema = 'mimiciv_hosp'
path = 'physionet.org/files/mimiciv/2.2/hosp/'

create_schema_and_tables(schema, path)

In [ ]:
# Import MIMIC-IV ICU
schema = 'mimiciv_icu'
path = 'physionet.org/files/mimiciv/2.2/icu/'

create_schema_and_tables(schema, path)

In [ ]:
# Import MIMIC-IV Notes
schema = 'mimiciv_note'
path = 'physionet.org/files/mimic-iv-note/2.2/note/'

create_schema_and_tables(schema, path)

In [ ]:
# # One-off fix for bad file
# schema = 'mimiciv_hosp'
# path = 'bad_files/'
# file = 'pharmacy.csv.gz'
# table = file.split('.')[0]
# param = col.get((schema, table), 'AUTO_DETECT=TRUE') + nullstr.get((schema, table), '')

# # %sql DROP TABLE IF EXISTS {schema}.{table};
# # %sql CREATE TABLE {schema}.{table} AS SELECT * FROM  read_csv('{path}{file}', header=True, {param} );

# create_schema_and_tables(schema, path)

In [ ]:
# Extract the column names and data_types for each table
# (from a manual inspection and load of the data)
# %sql columns << SELECT table_schema, table_name, column_name, data_type FROM information_schema.columns;

# columns.to_clipboard(index=False)
# print(columns)